# 単純な量子回路をゼロから書く

{doc}`第一回の前半 <chsh_inequality>`で、量子回路の基本的な書き方と実機での実行の仕方を学びました。そこで出てきたように、QCでは量子レジスタに特定の量子状態を実現することが計算を行うこととなります。

後半では、基本的なゲートを組み合わせて様々な量子状態を作る実習を行います。

```{contents} 目次
---
local: true
---
```

$\newcommand{ket}[1]{|#1\rangle}$

## シミュレータで量子状態を調べる

前半で見たように、QCで実現される量子状態は、量子力学の公理に基づいて理論的に計算・予測できます。そこで用いられる数学的操作も単なる足し算や掛け算（線形演算）なので、実はQCの量子状態は（古典）計算機で比較的簡単に計算できます。当然のことですが、QCは何も魔法のブラックボックスというわけではありません。

ただし、古典計算機で量子状態を再現するためには、特殊な場合を除いて、量子ビット数の指数関数的な量のメモリが必要になります。これも前半で見たように、$n$量子ビットあれば、系の自由度（degrees of freedom / d.o.f.: 実数自由パラメータの数）は$2^{n+1} - 2$ですので、例えば各自由度を64ビットの浮動小数点で表現するとしたら、必要なメモリは(-2を無視して)

$$
2^6\, \mathrm{(bits / d.o.f.)} \times 2^{n+1}\, \mathrm{(d.o.f.)} \times 2^{-3}\, \mathrm{(bytes / bit)} = 2^{n+4}\, \mathrm{(bytes)}
$$

なので、$n=16$で1 MiB、$n=26$で1 GiB、$n=36$で1 TiBです。現在の計算機では、よほどのハイエンドワークステーションでもRAMは$\mathcal{O}(1)$ TiBなのに対し、QCではすでに$n>50$のものが存在するので、後者が使いこなせて有用な問題を解くことができればいわゆる「量子超越」(quantum supremacy)が達成されたということになります。

話が逸れましたが、逆に言うと、$n \sim 30$程度までの回路であれば、ある程度のスペックを持った計算機で厳密にシミュレートできるということが言えます。じっさい世の中には[数多くの](https://quantiki.org/wiki/list-qc-simulators)シミュレータが存在します。

実機での計算に対するシミュレーションの利点として、

- **量子振幅が直接見える**：実機では回路を繰り返し実行・測定することで計算基底に対する確率分布（＝量子振幅の絶対値自乗）を推定することができますが、複素位相までを含めた振幅の情報を知ることはできません。
- **回路の途中の状態を調べられる**：実機でも回路を測定で中断することでその時点での状態をある程度調べることができますが、専用の回路を用意しなければいけません。
- **エラーがない**：前半の実習の最後で見たように、NISQの実機には様々なエラーがあり、計算結果が予測と大きくずれることがあります。シミュレータではエラーの影響を受けずに純粋に回路の状態を調べることができます。

などが挙げられます。

Qiskitにも様々な高機能シミュレータが同梱されているので、その使い方と出力の読み方を説明します。

まずは`Aer`というオブジェクトをインポートします。

In [1]:
from qiskit import Aer

`Aer`は前半に登場した`IBMQ`と同様の構造をしており、複数のシミュレータをバックエンドとして管理しています。量子振幅を直接観察するには、そのうちの`statevector_simulator`を使用します。

In [2]:
simulator = Aer.get_backend('statevector_simulator')

シミュレートする回路を作ります。ここでは例として前半で登場した回路1を使います。測定以外のゲートをかけていきます。

In [3]:
from qiskit import QuantumCircuit
import numpy as np

circuit = QuantumCircuit(2)
circuit.h(0)
circuit.cx(0, 1)
circuit.ry(3. * np.pi / 4., 1)

```{warning}
回路に測定操作(`measure_all()`)を加えてしまうと、`statevector_simulator`を利用して測定前の回路の量子状態を確認することができなくなります。
```

シミュレーションを実行する際も、実機のバックエンドと同様に`execute`関数を使い、ジョブオブジェクトから結果を得ます。ただし、今回は計算結果からカウントではなく「状態ベクトル」（＝量子振幅の配列）を得るので`get_counts`ではなく`result.data()['statevector']`を参照します。

In [4]:
from qiskit import execute
job = execute(circuit, simulator)
result = job.result()
statevector = result.data()['statevector']

print(type(statevector), statevector.dtype)
print(statevector)

<class 'numpy.ndarray'> complex128
[ 0.27059805+0.j -0.65328148+0.j  0.65328148+0.j  0.27059805+0.j]


状態ベクトルは[numpy](https://numpy.org/)の`ndarray`オブジェクトとして実装されています。データタイプは128ビットの複素数（実部と虚部それぞれ64ビット）です。アレイのインデックスがそのまま二進数としてみなした計算基底の値に対応しています。

In [5]:
from IPython.display import display, Markdown

state_string = r'$\ket{\psi} ='
for idx in range(statevector.shape[0]):
    angle = np.arcsin(statevector[idx].real * np.sqrt(2.))
    norm_angle = np.round(angle / np.pi * 8.).astype(int)
    state_string += r' \sin({} \pi / 8)\ket{{{}}}'.format('' if norm_angle == 1 else norm_angle, idx)
    if idx != statevector.shape[0] - 1:
        state_string += '+'

state_string += r'$'
display(Markdown(state_string))

$\ket{\psi} = \sin( \pi / 8)\ket{0}+ \sin(-3 \pi / 8)\ket{1}+ \sin(3 \pi / 8)\ket{2}+ \sin( \pi / 8)\ket{3}$

## その他のゲート

前半でよく使われる1量子ビットゲートと2量子ビット制御ゲートを紹介しましたが、他にも知っておくと便利なゲートがいくつかあります。

### 1、2量子ビット

```{list-table}
:header-rows: 1
* - ゲート名
  - 対象量子ビット数
  - 説明
  - Qiskitコード
* - $U_1$, $P$
  - 1
  - パラメータ$\phi$を取り、$\ket{1}$に位相$e^{i\phi}$をかける。$P(\phi)$は$R_{z}(\phi)$と等価なので、1量子ビットゲートとして利用する分にはどちらでも同じ結果が得られる。
  - `circuit.p(phi, i)`
* - $U_3$
  - 1
  - パラメータ$\theta, \phi, \lambda$を取り、単一量子ビットの任意の状態を実現する。
    ```{math}
    U_3(\theta, \phi, \lambda)\ket{0} = \cos\left(\frac{\theta}{2}\right) \ket{0} + e^{i\phi}\sin\left(\frac{\theta}{2}\right) \ket{1} \\
    U_3(\theta, \phi, \lambda)\ket{1} = -e^{i\lambda}\sin\left(\frac{\theta}{2}\right) \ket{0} + e^{i(\phi+\lambda)}\cos\left(\frac{\theta}{2}\right) \ket{1}
    ```
  - `circuit.u3(theta, phi, lambda, i)`
* - $C^i_j[U_1]$, $C^i_j[P]$
  - 2
  - パラメータ$\phi$を取り、ビット$i, j$が1であるような計算基底の位相を$\phi$前進させる。
    ```{math}
    C^i_j[U_1(\phi)] \ket{00} = \ket{00} \\
    C^i_j[U_1(\phi)] \ket{01} = \ket{01} \\
    C^i_j[U_1(\phi)] \ket{10} = \ket{10} \\
    C^i_j[U_1(\phi)] \ket{11} = e^{i\phi} \ket{11}
    ```
  - `circuit.cp(phi, i, j)`
* - SWAP
  - 2
  - 二つの量子ビットの量子状態を交換する。CNOT三回で実装できる。
    ```{math}
    \begin{align}
    & C^0_1[X]C^1_0[X]C^0_1[X] (\alpha \ket{00} + \beta \ket{01} + \gamma \ket{10} + \delta \ket{11}) \\
    = & C^0_1[X]C^1_0[X] (\alpha \ket{00} + \beta \ket{01} + \gamma \ket{11} + \delta \ket{10}) \\
    = & C^0_1[X] (\alpha \ket{00} + \beta \ket{11} + \gamma \ket{01} + \delta \ket{10}) \\
    = & \alpha \ket{00} + \beta \ket{10} + \gamma \ket{01} + \delta \ket{11}
    \end{align}
    ```
  - `circuit.swap(i, j)`
```

### 3量子ビット以上

量子レジスタに対する全ての操作が1量子ビットゲートと2量子ビット制御ゲート（実際にはCNOTのみ）の組み合わせで表現できることはすでに触れましたが、実際に量子回路を書くときには3つ以上の量子ビットにかかるゲートを使用したほうがアルゴリズムをコンパクトに表現できます。そのようなゲートを含む回路を実機で実行するときには、「トランスパイル」（transpilation）という過程を経て最終的には全て1量子ビットゲートとCNOTに自動的に分解されます。

3量子ビット以上にかかるゲートも基本は制御ゲートで、よく使われるのは制御ビットが複数あるような多重制御（multi-controlled）ゲート$C^{ij\dots}_{k}[U]$です。そのうち制御ビットが2つで標的ビット1つに$X$をかける$C^{ij}_{k}[X]$は特別多用され、Toffoliゲートとも呼ばれます。

多重制御ゲートでない3量子ビットゲートの例として、controlled SWAPというものも存在します。制御ビット1つで他の2つの量子ビットの間の量子状態交換を制御します。当然制御ビットを増やしたmulti-controlled SWAPというものも考えられます。

Qiskitで多重制御ゲートに対応するコードは一般に`circuit.mc?(parameters, [controls], target)`という形を取ります。例えばToffoliゲートは

```{code-block} python
circuit.mcx([i, j], k)
```

で、$C^{ij}_{k}[R_y]$は

```{code-block} python
circuit.mcry(theta, [i, j], k)
```

となります。

## 量子状態生成

それでは、これまでに登場したゲートを使って量子状態生成のエクササイズをしましょう。

まずは回路の量子状態を表示する`show_state`という関数を定義します。

In [20]:
def show_state(circuit, amp_norm=None, phase_norm=(np.pi, '\pi')):
    """Print the quantum state of the circuit in latex markdown.
    
    Args:
        circuit (QuantumCircuit): The circuit.
        amp_norm (None or tuple): If not None, specify the normalization of the amplitudes by
            (numeric devisor, unit in latex)
        phase_norm (None or tuple): If not None, specify the normalization of the phases by
            (numeric devisor, unit in latex)
    """

    # Run the circuit in statevector_simulator and obtain the final state statevector
    simulator = Aer.get_backend('statevector_simulator')
    statevector = execute(circuit, simulator).result().data()['statevector']

    # Absolute value and the phase of the amplitudes
    absamp = np.abs(statevector)
    logamp = np.zeros_like(statevector)
    np.log(statevector, out=logamp, where=(absamp > 0.))
    phase = logamp.imag
    
    # Set the numerical tolerance for various comparisons
    tolerance = 1.e-3
        
    # Final latex string
    state = ''
    
    # Sign of each term (initialized to 0 so the first term doesn't get a + in front)
    sign = 0
    
    for idx in range(statevector.shape[0]):
        a = absamp[idx]
        p = phase[idx]
        
        # Skip terms with absolute amplitude less than tolerance
        if a < tolerance:
            continue

        # Latex string for this term
        basis_term = ''

        # Write the amplitude
        if amp_norm is None:
            # No amplitude normalization -> just write as a raw float
            basis_term += r'{:.2f}'.format(a)
        else:
            # With amplitude normalization
            a /= amp_norm[0]
            
            if abs(a - np.round(a)) < tolerance:
                # Amplitude is integer * norm
                if np.round(a) != 1.:
                    basis_term += r'{:d} \cdot '.format(np.round(a).astype(int))
            else:
                # Otherwise float * norm
                basis_term += r'{:.2f} \cdot '.format(a)
                
            basis_term += amp_norm[1]

        # Write the phase
        if abs(p) > tolerance:
            while p < 0.:
                p += 2. * np.pi
            
            # First check if the phase is a multiple of pi or pi/2
            pbypi = p / np.pi
            if abs(pbypi - np.round(pbypi)) < tolerance:
                # Multiple of pi
                pbypi = abs(np.round(pbypi).astype(int))
                if pbypi % 2 == 1:
                    sign = -1
            elif abs(2. * pbypi - np.round(2. * pbypi)) < tolerance:
                # Multiple of pi/2
                twopbypi = np.round(2. * pbypi).astype(int)
                if twopbypi % 2 == 1:
                    basis_term += 'i'
                if twopbypi % 4 == 3:
                    sign = -1
            else:
                basis_term += r'e^{'
                if phase_norm is None:
                    # No phase normalization -> write as exp(raw float * i)
                    basis_term += '{:.2f}'.format(p)
                else:
                    # With phase normalization -> write as exp(divident * norm * i)
                    p /= phase_norm[0]
                    
                    if abs(p - np.round(p)) < tolerance:
                        if np.round(p) != 1.:
                            basis_term += r'{:d} \cdot '.format(np.round(p).astype(int))
                    else:
                        basis_term += r'{:.2f} \cdot '.format(p)

                    basis_term += phase_norm[1]

                basis_term += r' i}'
                
        basis_term += r' |{}\rangle'.format(idx)

        if sign == 1:
            state += ' + '
        elif sign == -1:
            state += ' - '
            
        state += basis_term
        
        sign = 1
        
    display(Markdown('${}$'.format(state)))

### 問題1: 1量子ビット、相対位相付き

**問題**

1量子ビットに対して状態

$$
\frac{1}{\sqrt{2}}\left(\ket{0} + i\ket{1}\right)
$$

を作りなさい。

In [7]:
circuit = QuantumCircuit(1)
##################
### EDIT BELOW ###
##################
circuit.h(0)
# circuit.?
##################
### EDIT ABOVE ###
##################

show_state(circuit, amp_norm=(np.sqrt(0.5), r'\frac{1}{\sqrt{2}}'))

$\frac{1}{\sqrt{2}} |0\rangle + \frac{1}{\sqrt{2}} |1\rangle$

**解答**

正解は無限に存在しますが、上の回路ですでにアダマールゲートがかかっているので、最も手っ取り早いのは

```{code-block} python
circuit.h(0)
circuit.p(np.pi / 2., 0)
```

でしょう。自分で入力して確認してみてください。

また、上で触れたように、単一量子ビットの場合、$U_3$ゲートを使えば任意の状態を一つのゲート操作で生成できます。$U_3$を使った解答は

```{code-block} python
circuit.u3(np.pi / 2., np.pi / 2., 0., 0)
```

です。

### 問題2: ベル状態、相対位相付き

**問題**

2量子ビットに対して状態

$$
\frac{1}{\sqrt{2}}\left(\ket{0} + i\ket{3}\right)
$$

を作りなさい。

In [8]:
circuit = QuantumCircuit(2)

##################
### EDIT BELOW ###
##################
# circuit.?
circuit.cx(0, 1)
##################
### EDIT ABOVE ###
##################

show_state(circuit, amp_norm=(np.sqrt(0.5), r'\frac{1}{\sqrt{2}}'))

$1.41 \cdot \frac{1}{\sqrt{2}} |0\rangle$

**解答**

問題1の結果を利用すると簡単です。初期状態は両ビットとも$\ket{0}$なので、2ビットレジスタに対して問題1の操作をすると状態$1/\frac{2}(\ket{0} + i\ket{1})\ket{0}$ができます。ここでビット0から1にCNOTをかけると、$\ket{1}\ket{0} \rightarrow \ket{1}\ket{1}$となるので、望む状態が実現します。

```{code-block} python
circuit.h(0)
circuit.p(np.pi / 2., 0)
circuit.cx(0, 1)
```

### 問題3: GHZ状態

**問題**

3量子ビットに対して状態

$$
\frac{1}{\sqrt{2}} (\ket{0} + \ket{7})
$$

を作りなさい。

In [9]:
circuit = QuantumCircuit(3)

##################
### EDIT BELOW ###
##################
# circuit.?
circuit.mcx([0, 1], 2)
##################
### EDIT ABOVE ###
##################

show_state(circuit, amp_norm=(np.sqrt(0.5), r'\frac{1}{\sqrt{2}}'))

$1.41 \cdot \frac{1}{\sqrt{2}} |0\rangle$

**解答**

Bell状態の生成と発想は同じですが、多重制御ゲート$C^{01}_2[X]$を利用します。

```{code-block} python
circuit.h(0)
circuit.cx(0, 1)
circuit.mcx([0, 1], 2)
```

### 問題4: Equal superposition

**問題**

一般の$n$量子ビットに対して状態

$$
\frac{1}{\sqrt{2^n}} \sum_{k=0}^{2^n-1} \ket{k}
$$

を作る回路を考え、$n=4$のケースを実装しなさい。

In [14]:
num_qubits = 4

circuit = QuantumCircuit(num_qubits)

##################
### EDIT BELOW ###
##################
# circuit.?
##################
### EDIT ABOVE ###
##################

sqrt_2_to_n = np.sqrt(2 ** num_qubits)
show_state(circuit, amp_norm=(1. / sqrt_2_to_n, r'\frac{1}{' + str(int(sqrt_2_to_n)) + '}'))

$4 \cdot \frac{1}{4} |0\rangle$

**解答**

Equal superpositionと呼ばれるこの状態は、量子計算でおそらく一番多様される状態です。一見複雑に思えますが、$n$桁の二進数で$0$から$2^n-1$までの数を表現すると、全ての0/1の組み合わせが登場するということを考えると、

$$
\frac{1}{\sqrt{2^n}} \sum_{k=0}^{2^n-1} \ket{k} = \frac{1}{\sqrt{2}}(\ket{0} + \ket{1}) \otimes \frac{1}{\sqrt{2}}(\ket{0} + \ket{1}) \otimes \cdots \otimes \frac{1}{\sqrt{2}}(\ket{0} + \ket{1})
$$

と、$n$個の$1/\sqrt{2}(\ket{0} + \ket{1})$の積と等しいことがわかります。つまり各ビットに対してアダマールゲートをかけた状態にほかならず、

```{code-block} python
for i in range(4):
    circuit.h(i)
```

が正解です。

### 問題5: 特定の基底の符号を反転させる

**問題**

問題4の4ビットequal superposition状態において、基底$\ket{5}$の符号を反転させなさい。

In [ ]:
num_qubits = 4

circuit = QuantumCircuit(num_qubits)

##################
### EDIT BELOW ###
##################
for i in range(num_qubits):
    circuit.h(i)
    
# circuit.?
##################
### EDIT ABOVE ###
##################

sqrt_2_to_n = np.sqrt(2 ** num_qubits)
show_state(circuit, amp_norm=(1. / sqrt_2_to_n, r'\frac{1}{' + str(int(sqrt_2_to_n)) + '}'))

**解答**

$\ket{5}=\ket{0101}$の符号を反転させるには、4つの量子ビットが0, 1, 0, 1という値を取っている状態のみに作用するゲートを使います。必然的に多重制御ゲートが用いられますが、残念ながら制御ゲートはビットの値が1であるときにしか働きません。そこで、まず$\ket{0101}$を$\ket{1111}$に変換し、その上で$C^{012}_{3}[Z]$を使います[^mcz]。符号を反転させたら、最初の操作の逆操作をして$\ket{1111}$を$\ket{0101}$に変換します。

```{code-block} python
for i in range(4):
    circuit.h(i)

circuit.x(1)
circuit.x(3)
circuit.mcz([0, 1, 2], 3)
circuit.x(1) # Xの逆操作はX
circuit.x(3)
```

ビット1と3にかかっている$X$ゲートは当然他の基底にも影響するので、例えば$\ket{0011}$は$\ket{0110}$となります。しかし、1と3に$X$をかけて$\ket{1111}$になるのは$\ket{0101}$だけなので、逆操作でちゃんと「後片付け」さえすれば、この一連の操作で$\ket{5}$だけをいじることができます。

[^mcz]: （多重）制御$Z$ゲートはどのビットを制御と呼んでも標的と呼んでも同じ作用をするので、$C^{123}_0[Z]$でもなんでも構いません。

### 問題6: Equal superpositionに位相を付ける

**問題**

一般の$n$量子ビットに対して状態

$$
\frac{1}{\sqrt{2^n}}\sum_{k=0}^{2^n-1} e^{2\pi i jk/2^n} \ket{k} \quad (j \in \mathbb{N})
$$

を作る回路を考え、$n=8, j=3$のケースを実装しなさい。

In [21]:
num_qubits = 8

circuit = QuantumCircuit(num_qubits)

j = 3

##################
### EDIT BELOW ###
##################
#for i in range(num_qubits):
#    circuit.h(i)
    
# circuit.?

for m in range(num_qubits):
    circuit.h(m)
    circuit.p(2. * np.pi * j / (2 ** (num_qubits - m)), m)
##################
### EDIT ABOVE ###
##################

sqrt_2_to_n = np.sqrt(2 ** num_qubits)
show_state(circuit, amp_norm=(1. / sqrt_2_to_n, r'\frac{1}{' + str(int(sqrt_2_to_n)) + '}'), phase_norm=(2 * np.pi * j / (2 ** num_qubits), r'\frac{2 \pi i j}{2^n}'))

[ 0.          0.07363108  0.14726216  0.22089323  0.29452431  0.36815539
  0.44178647  0.51541754  0.58904862  0.6626797   0.73631078  0.80994186
  0.88357293  0.95720401  1.03083509  1.10446617  1.17809725  1.25172832
  1.3253594   1.39899048  1.47262156  1.54625263  1.61988371  1.69351479
  1.76714587  1.84077695  1.91440802  1.9880391   2.06167018  2.13530126
  2.20893233  2.28256341  2.35619449  2.42982557  2.50345665  2.57708772
  2.6507188   2.72434988  2.79798096  2.87161203  2.94524311  3.01887419
  3.09250527 -3.11704896 -3.04341788 -2.96978681 -2.89615573 -2.82252465
 -2.74889357 -2.67526249 -2.60163142 -2.52800034 -2.45436926 -2.38073818
 -2.3071071  -2.23347603 -2.15984495 -2.08621387 -2.01258279 -1.93895172
 -1.86532064 -1.79168956 -1.71805848 -1.6444274  -1.57079633 -1.49716525
 -1.42353417 -1.34990309 -1.27627202 -1.20264094 -1.12900986 -1.05537878
 -0.9817477  -0.90811663 -0.83448555 -0.76085447 -0.68722339 -0.61359232
 -0.53996124 -0.46633016 -0.39269908 -0.319068   -0

$\frac{1}{16} |0\rangle + \frac{1}{16}e^{\frac{2 \pi i j}{2^n} i} |1\rangle + \frac{1}{16}e^{2 \cdot \frac{2 \pi i j}{2^n} i} |2\rangle + \frac{1}{16}e^{3 \cdot \frac{2 \pi i j}{2^n} i} |3\rangle + \frac{1}{16}e^{4 \cdot \frac{2 \pi i j}{2^n} i} |4\rangle + \frac{1}{16}e^{5 \cdot \frac{2 \pi i j}{2^n} i} |5\rangle + \frac{1}{16}e^{6 \cdot \frac{2 \pi i j}{2^n} i} |6\rangle + \frac{1}{16}e^{7 \cdot \frac{2 \pi i j}{2^n} i} |7\rangle + \frac{1}{16}e^{8 \cdot \frac{2 \pi i j}{2^n} i} |8\rangle + \frac{1}{16}e^{9 \cdot \frac{2 \pi i j}{2^n} i} |9\rangle + \frac{1}{16}e^{10 \cdot \frac{2 \pi i j}{2^n} i} |10\rangle + \frac{1}{16}e^{11 \cdot \frac{2 \pi i j}{2^n} i} |11\rangle + \frac{1}{16}e^{12 \cdot \frac{2 \pi i j}{2^n} i} |12\rangle + \frac{1}{16}e^{13 \cdot \frac{2 \pi i j}{2^n} i} |13\rangle + \frac{1}{16}e^{14 \cdot \frac{2 \pi i j}{2^n} i} |14\rangle + \frac{1}{16}e^{15 \cdot \frac{2 \pi i j}{2^n} i} |15\rangle + \frac{1}{16}e^{16 \cdot \frac{2 \pi i j}{2^n} i} |16\rangle + \frac{1}{16}e^{17 \cdot \frac{2 \pi i j}{2^n} i} |17\rangle + \frac{1}{16}e^{18 \cdot \frac{2 \pi i j}{2^n} i} |18\rangle + \frac{1}{16}e^{19 \cdot \frac{2 \pi i j}{2^n} i} |19\rangle + \frac{1}{16}e^{20 \cdot \frac{2 \pi i j}{2^n} i} |20\rangle + \frac{1}{16}e^{21 \cdot \frac{2 \pi i j}{2^n} i} |21\rangle + \frac{1}{16}e^{22 \cdot \frac{2 \pi i j}{2^n} i} |22\rangle + \frac{1}{16}e^{23 \cdot \frac{2 \pi i j}{2^n} i} |23\rangle + \frac{1}{16}e^{24 \cdot \frac{2 \pi i j}{2^n} i} |24\rangle + \frac{1}{16}e^{25 \cdot \frac{2 \pi i j}{2^n} i} |25\rangle + \frac{1}{16}e^{26 \cdot \frac{2 \pi i j}{2^n} i} |26\rangle + \frac{1}{16}e^{27 \cdot \frac{2 \pi i j}{2^n} i} |27\rangle + \frac{1}{16}e^{28 \cdot \frac{2 \pi i j}{2^n} i} |28\rangle + \frac{1}{16}e^{29 \cdot \frac{2 \pi i j}{2^n} i} |29\rangle + \frac{1}{16}e^{30 \cdot \frac{2 \pi i j}{2^n} i} |30\rangle + \frac{1}{16}e^{31 \cdot \frac{2 \pi i j}{2^n} i} |31\rangle + \frac{1}{16}e^{32 \cdot \frac{2 \pi i j}{2^n} i} |32\rangle + \frac{1}{16}e^{33 \cdot \frac{2 \pi i j}{2^n} i} |33\rangle + \frac{1}{16}e^{34 \cdot \frac{2 \pi i j}{2^n} i} |34\rangle + \frac{1}{16}e^{35 \cdot \frac{2 \pi i j}{2^n} i} |35\rangle + \frac{1}{16}e^{36 \cdot \frac{2 \pi i j}{2^n} i} |36\rangle + \frac{1}{16}e^{37 \cdot \frac{2 \pi i j}{2^n} i} |37\rangle + \frac{1}{16}e^{38 \cdot \frac{2 \pi i j}{2^n} i} |38\rangle + \frac{1}{16}e^{39 \cdot \frac{2 \pi i j}{2^n} i} |39\rangle + \frac{1}{16}e^{40 \cdot \frac{2 \pi i j}{2^n} i} |40\rangle + \frac{1}{16}e^{41 \cdot \frac{2 \pi i j}{2^n} i} |41\rangle + \frac{1}{16}e^{42 \cdot \frac{2 \pi i j}{2^n} i} |42\rangle + \frac{1}{16}e^{43 \cdot \frac{2 \pi i j}{2^n} i} |43\rangle + \frac{1}{16}e^{44 \cdot \frac{2 \pi i j}{2^n} i} |44\rangle + \frac{1}{16}e^{45 \cdot \frac{2 \pi i j}{2^n} i} |45\rangle + \frac{1}{16}e^{46 \cdot \frac{2 \pi i j}{2^n} i} |46\rangle + \frac{1}{16}e^{47 \cdot \frac{2 \pi i j}{2^n} i} |47\rangle + \frac{1}{16}e^{48 \cdot \frac{2 \pi i j}{2^n} i} |48\rangle + \frac{1}{16}e^{49 \cdot \frac{2 \pi i j}{2^n} i} |49\rangle + \frac{1}{16}e^{50 \cdot \frac{2 \pi i j}{2^n} i} |50\rangle + \frac{1}{16}e^{51 \cdot \frac{2 \pi i j}{2^n} i} |51\rangle + \frac{1}{16}e^{52 \cdot \frac{2 \pi i j}{2^n} i} |52\rangle + \frac{1}{16}e^{53 \cdot \frac{2 \pi i j}{2^n} i} |53\rangle + \frac{1}{16}e^{54 \cdot \frac{2 \pi i j}{2^n} i} |54\rangle + \frac{1}{16}e^{55 \cdot \frac{2 \pi i j}{2^n} i} |55\rangle + \frac{1}{16}e^{56 \cdot \frac{2 \pi i j}{2^n} i} |56\rangle + \frac{1}{16}e^{57 \cdot \frac{2 \pi i j}{2^n} i} |57\rangle + \frac{1}{16}e^{58 \cdot \frac{2 \pi i j}{2^n} i} |58\rangle + \frac{1}{16}e^{59 \cdot \frac{2 \pi i j}{2^n} i} |59\rangle + \frac{1}{16}e^{60 \cdot \frac{2 \pi i j}{2^n} i} |60\rangle + \frac{1}{16}e^{61 \cdot \frac{2 \pi i j}{2^n} i} |61\rangle + \frac{1}{16}e^{62 \cdot \frac{2 \pi i j}{2^n} i} |62\rangle + \frac{1}{16}e^{63 \cdot \frac{2 \pi i j}{2^n} i} |63\rangle - \frac{1}{16}i |64\rangle + \frac{1}{16}e^{65 \cdot \frac{2 \pi i j}{2^n} i} |65\rangle + \frac{1}{16}e^{66 \cdot \frac{2 \pi i j}{2^n} i} |66\rangle + \frac{1}{16}e^{67 \cdot \frac{2 \pi i j}{2^n} i} |67\rangle + \frac{1}{16}e^{68 \cdot \frac{2 \pi i j}{2^n} i} |68\rangle + \frac{1}{16}e^{69 \cdot \frac{2 \pi i j}{2^n} i} |69\rangle + \frac{1}{16}e^{70 \cdot \frac{2 \pi i j}{2^n} i} |70\rangle + \frac{1}{16}e^{71 \cdot \frac{2 \pi i j}{2^n} i} |71\rangle + \frac{1}{16}e^{72 \cdot \frac{2 \pi i j}{2^n} i} |72\rangle + \frac{1}{16}e^{73 \cdot \frac{2 \pi i j}{2^n} i} |73\rangle + \frac{1}{16}e^{74 \cdot \frac{2 \pi i j}{2^n} i} |74\rangle + \frac{1}{16}e^{75 \cdot \frac{2 \pi i j}{2^n} i} |75\rangle + \frac{1}{16}e^{76 \cdot \frac{2 \pi i j}{2^n} i} |76\rangle + \frac{1}{16}e^{77 \cdot \frac{2 \pi i j}{2^n} i} |77\rangle + \frac{1}{16}e^{78 \cdot \frac{2 \pi i j}{2^n} i} |78\rangle + \frac{1}{16}e^{79 \cdot \frac{2 \pi i j}{2^n} i} |79\rangle + \frac{1}{16}e^{80 \cdot \frac{2 \pi i j}{2^n} i} |80\rangle + \frac{1}{16}e^{81 \cdot \frac{2 \pi i j}{2^n} i} |81\rangle + \frac{1}{16}e^{82 \cdot \frac{2 \pi i j}{2^n} i} |82\rangle + \frac{1}{16}e^{83 \cdot \frac{2 \pi i j}{2^n} i} |83\rangle + \frac{1}{16}e^{84 \cdot \frac{2 \pi i j}{2^n} i} |84\rangle + \frac{1}{16}e^{85 \cdot \frac{2 \pi i j}{2^n} i} |85\rangle + \frac{1}{16}e^{0.67 \cdot \frac{2 \pi i j}{2^n} i} |86\rangle + \frac{1}{16}e^{1.67 \cdot \frac{2 \pi i j}{2^n} i} |87\rangle + \frac{1}{16}e^{2.67 \cdot \frac{2 \pi i j}{2^n} i} |88\rangle + \frac{1}{16}e^{3.67 \cdot \frac{2 \pi i j}{2^n} i} |89\rangle + \frac{1}{16}e^{4.67 \cdot \frac{2 \pi i j}{2^n} i} |90\rangle + \frac{1}{16}e^{5.67 \cdot \frac{2 \pi i j}{2^n} i} |91\rangle + \frac{1}{16}e^{6.67 \cdot \frac{2 \pi i j}{2^n} i} |92\rangle + \frac{1}{16}e^{7.67 \cdot \frac{2 \pi i j}{2^n} i} |93\rangle + \frac{1}{16}e^{8.67 \cdot \frac{2 \pi i j}{2^n} i} |94\rangle + \frac{1}{16}e^{9.67 \cdot \frac{2 \pi i j}{2^n} i} |95\rangle + \frac{1}{16}e^{10.67 \cdot \frac{2 \pi i j}{2^n} i} |96\rangle + \frac{1}{16}e^{11.67 \cdot \frac{2 \pi i j}{2^n} i} |97\rangle + \frac{1}{16}e^{12.67 \cdot \frac{2 \pi i j}{2^n} i} |98\rangle + \frac{1}{16}e^{13.67 \cdot \frac{2 \pi i j}{2^n} i} |99\rangle + \frac{1}{16}e^{14.67 \cdot \frac{2 \pi i j}{2^n} i} |100\rangle + \frac{1}{16}e^{15.67 \cdot \frac{2 \pi i j}{2^n} i} |101\rangle + \frac{1}{16}e^{16.67 \cdot \frac{2 \pi i j}{2^n} i} |102\rangle + \frac{1}{16}e^{17.67 \cdot \frac{2 \pi i j}{2^n} i} |103\rangle + \frac{1}{16}e^{18.67 \cdot \frac{2 \pi i j}{2^n} i} |104\rangle + \frac{1}{16}e^{19.67 \cdot \frac{2 \pi i j}{2^n} i} |105\rangle + \frac{1}{16}e^{20.67 \cdot \frac{2 \pi i j}{2^n} i} |106\rangle + \frac{1}{16}e^{21.67 \cdot \frac{2 \pi i j}{2^n} i} |107\rangle + \frac{1}{16}e^{22.67 \cdot \frac{2 \pi i j}{2^n} i} |108\rangle + \frac{1}{16}e^{23.67 \cdot \frac{2 \pi i j}{2^n} i} |109\rangle + \frac{1}{16}e^{24.67 \cdot \frac{2 \pi i j}{2^n} i} |110\rangle + \frac{1}{16}e^{25.67 \cdot \frac{2 \pi i j}{2^n} i} |111\rangle + \frac{1}{16}e^{26.67 \cdot \frac{2 \pi i j}{2^n} i} |112\rangle + \frac{1}{16}e^{27.67 \cdot \frac{2 \pi i j}{2^n} i} |113\rangle + \frac{1}{16}e^{28.67 \cdot \frac{2 \pi i j}{2^n} i} |114\rangle + \frac{1}{16}e^{29.67 \cdot \frac{2 \pi i j}{2^n} i} |115\rangle + \frac{1}{16}e^{30.67 \cdot \frac{2 \pi i j}{2^n} i} |116\rangle + \frac{1}{16}e^{31.67 \cdot \frac{2 \pi i j}{2^n} i} |117\rangle + \frac{1}{16}e^{32.67 \cdot \frac{2 \pi i j}{2^n} i} |118\rangle + \frac{1}{16}e^{33.67 \cdot \frac{2 \pi i j}{2^n} i} |119\rangle + \frac{1}{16}e^{34.67 \cdot \frac{2 \pi i j}{2^n} i} |120\rangle + \frac{1}{16}e^{35.67 \cdot \frac{2 \pi i j}{2^n} i} |121\rangle + \frac{1}{16}e^{36.67 \cdot \frac{2 \pi i j}{2^n} i} |122\rangle + \frac{1}{16}e^{37.67 \cdot \frac{2 \pi i j}{2^n} i} |123\rangle + \frac{1}{16}e^{38.67 \cdot \frac{2 \pi i j}{2^n} i} |124\rangle + \frac{1}{16}e^{39.67 \cdot \frac{2 \pi i j}{2^n} i} |125\rangle + \frac{1}{16}e^{40.67 \cdot \frac{2 \pi i j}{2^n} i} |126\rangle + \frac{1}{16}e^{41.67 \cdot \frac{2 \pi i j}{2^n} i} |127\rangle - \frac{1}{16} |128\rangle + \frac{1}{16}e^{43.67 \cdot \frac{2 \pi i j}{2^n} i} |129\rangle + \frac{1}{16}e^{44.67 \cdot \frac{2 \pi i j}{2^n} i} |130\rangle + \frac{1}{16}e^{45.67 \cdot \frac{2 \pi i j}{2^n} i} |131\rangle + \frac{1}{16}e^{46.67 \cdot \frac{2 \pi i j}{2^n} i} |132\rangle + \frac{1}{16}e^{47.67 \cdot \frac{2 \pi i j}{2^n} i} |133\rangle + \frac{1}{16}e^{48.67 \cdot \frac{2 \pi i j}{2^n} i} |134\rangle + \frac{1}{16}e^{49.67 \cdot \frac{2 \pi i j}{2^n} i} |135\rangle + \frac{1}{16}e^{50.67 \cdot \frac{2 \pi i j}{2^n} i} |136\rangle + \frac{1}{16}e^{51.67 \cdot \frac{2 \pi i j}{2^n} i} |137\rangle + \frac{1}{16}e^{52.67 \cdot \frac{2 \pi i j}{2^n} i} |138\rangle + \frac{1}{16}e^{53.67 \cdot \frac{2 \pi i j}{2^n} i} |139\rangle + \frac{1}{16}e^{54.67 \cdot \frac{2 \pi i j}{2^n} i} |140\rangle + \frac{1}{16}e^{55.67 \cdot \frac{2 \pi i j}{2^n} i} |141\rangle + \frac{1}{16}e^{56.67 \cdot \frac{2 \pi i j}{2^n} i} |142\rangle + \frac{1}{16}e^{57.67 \cdot \frac{2 \pi i j}{2^n} i} |143\rangle + \frac{1}{16}e^{58.67 \cdot \frac{2 \pi i j}{2^n} i} |144\rangle + \frac{1}{16}e^{59.67 \cdot \frac{2 \pi i j}{2^n} i} |145\rangle + \frac{1}{16}e^{60.67 \cdot \frac{2 \pi i j}{2^n} i} |146\rangle + \frac{1}{16}e^{61.67 \cdot \frac{2 \pi i j}{2^n} i} |147\rangle + \frac{1}{16}e^{62.67 \cdot \frac{2 \pi i j}{2^n} i} |148\rangle + \frac{1}{16}e^{63.67 \cdot \frac{2 \pi i j}{2^n} i} |149\rangle + \frac{1}{16}e^{64.67 \cdot \frac{2 \pi i j}{2^n} i} |150\rangle + \frac{1}{16}e^{65.67 \cdot \frac{2 \pi i j}{2^n} i} |151\rangle + \frac{1}{16}e^{66.67 \cdot \frac{2 \pi i j}{2^n} i} |152\rangle + \frac{1}{16}e^{67.67 \cdot \frac{2 \pi i j}{2^n} i} |153\rangle + \frac{1}{16}e^{68.67 \cdot \frac{2 \pi i j}{2^n} i} |154\rangle + \frac{1}{16}e^{69.67 \cdot \frac{2 \pi i j}{2^n} i} |155\rangle + \frac{1}{16}e^{70.67 \cdot \frac{2 \pi i j}{2^n} i} |156\rangle + \frac{1}{16}e^{71.67 \cdot \frac{2 \pi i j}{2^n} i} |157\rangle + \frac{1}{16}e^{72.67 \cdot \frac{2 \pi i j}{2^n} i} |158\rangle + \frac{1}{16}e^{73.67 \cdot \frac{2 \pi i j}{2^n} i} |159\rangle + \frac{1}{16}e^{74.67 \cdot \frac{2 \pi i j}{2^n} i} |160\rangle + \frac{1}{16}e^{75.67 \cdot \frac{2 \pi i j}{2^n} i} |161\rangle + \frac{1}{16}e^{76.67 \cdot \frac{2 \pi i j}{2^n} i} |162\rangle + \frac{1}{16}e^{77.67 \cdot \frac{2 \pi i j}{2^n} i} |163\rangle + \frac{1}{16}e^{78.67 \cdot \frac{2 \pi i j}{2^n} i} |164\rangle + \frac{1}{16}e^{79.67 \cdot \frac{2 \pi i j}{2^n} i} |165\rangle + \frac{1}{16}e^{80.67 \cdot \frac{2 \pi i j}{2^n} i} |166\rangle + \frac{1}{16}e^{81.67 \cdot \frac{2 \pi i j}{2^n} i} |167\rangle + \frac{1}{16}e^{82.67 \cdot \frac{2 \pi i j}{2^n} i} |168\rangle + \frac{1}{16}e^{83.67 \cdot \frac{2 \pi i j}{2^n} i} |169\rangle + \frac{1}{16}e^{84.67 \cdot \frac{2 \pi i j}{2^n} i} |170\rangle + \frac{1}{16}e^{0.33 \cdot \frac{2 \pi i j}{2^n} i} |171\rangle + \frac{1}{16}e^{1.33 \cdot \frac{2 \pi i j}{2^n} i} |172\rangle + \frac{1}{16}e^{2.33 \cdot \frac{2 \pi i j}{2^n} i} |173\rangle + \frac{1}{16}e^{3.33 \cdot \frac{2 \pi i j}{2^n} i} |174\rangle + \frac{1}{16}e^{4.33 \cdot \frac{2 \pi i j}{2^n} i} |175\rangle + \frac{1}{16}e^{5.33 \cdot \frac{2 \pi i j}{2^n} i} |176\rangle + \frac{1}{16}e^{6.33 \cdot \frac{2 \pi i j}{2^n} i} |177\rangle + \frac{1}{16}e^{7.33 \cdot \frac{2 \pi i j}{2^n} i} |178\rangle + \frac{1}{16}e^{8.33 \cdot \frac{2 \pi i j}{2^n} i} |179\rangle + \frac{1}{16}e^{9.33 \cdot \frac{2 \pi i j}{2^n} i} |180\rangle + \frac{1}{16}e^{10.33 \cdot \frac{2 \pi i j}{2^n} i} |181\rangle + \frac{1}{16}e^{11.33 \cdot \frac{2 \pi i j}{2^n} i} |182\rangle + \frac{1}{16}e^{12.33 \cdot \frac{2 \pi i j}{2^n} i} |183\rangle + \frac{1}{16}e^{13.33 \cdot \frac{2 \pi i j}{2^n} i} |184\rangle + \frac{1}{16}e^{14.33 \cdot \frac{2 \pi i j}{2^n} i} |185\rangle + \frac{1}{16}e^{15.33 \cdot \frac{2 \pi i j}{2^n} i} |186\rangle + \frac{1}{16}e^{16.33 \cdot \frac{2 \pi i j}{2^n} i} |187\rangle + \frac{1}{16}e^{17.33 \cdot \frac{2 \pi i j}{2^n} i} |188\rangle + \frac{1}{16}e^{18.33 \cdot \frac{2 \pi i j}{2^n} i} |189\rangle + \frac{1}{16}e^{19.33 \cdot \frac{2 \pi i j}{2^n} i} |190\rangle + \frac{1}{16}e^{20.33 \cdot \frac{2 \pi i j}{2^n} i} |191\rangle + \frac{1}{16}i |192\rangle + \frac{1}{16}e^{22.33 \cdot \frac{2 \pi i j}{2^n} i} |193\rangle + \frac{1}{16}e^{23.33 \cdot \frac{2 \pi i j}{2^n} i} |194\rangle + \frac{1}{16}e^{24.33 \cdot \frac{2 \pi i j}{2^n} i} |195\rangle + \frac{1}{16}e^{25.33 \cdot \frac{2 \pi i j}{2^n} i} |196\rangle + \frac{1}{16}e^{26.33 \cdot \frac{2 \pi i j}{2^n} i} |197\rangle + \frac{1}{16}e^{27.33 \cdot \frac{2 \pi i j}{2^n} i} |198\rangle + \frac{1}{16}e^{28.33 \cdot \frac{2 \pi i j}{2^n} i} |199\rangle + \frac{1}{16}e^{29.33 \cdot \frac{2 \pi i j}{2^n} i} |200\rangle + \frac{1}{16}e^{30.33 \cdot \frac{2 \pi i j}{2^n} i} |201\rangle + \frac{1}{16}e^{31.33 \cdot \frac{2 \pi i j}{2^n} i} |202\rangle + \frac{1}{16}e^{32.33 \cdot \frac{2 \pi i j}{2^n} i} |203\rangle + \frac{1}{16}e^{33.33 \cdot \frac{2 \pi i j}{2^n} i} |204\rangle + \frac{1}{16}e^{34.33 \cdot \frac{2 \pi i j}{2^n} i} |205\rangle + \frac{1}{16}e^{35.33 \cdot \frac{2 \pi i j}{2^n} i} |206\rangle + \frac{1}{16}e^{36.33 \cdot \frac{2 \pi i j}{2^n} i} |207\rangle + \frac{1}{16}e^{37.33 \cdot \frac{2 \pi i j}{2^n} i} |208\rangle + \frac{1}{16}e^{38.33 \cdot \frac{2 \pi i j}{2^n} i} |209\rangle + \frac{1}{16}e^{39.33 \cdot \frac{2 \pi i j}{2^n} i} |210\rangle + \frac{1}{16}e^{40.33 \cdot \frac{2 \pi i j}{2^n} i} |211\rangle + \frac{1}{16}e^{41.33 \cdot \frac{2 \pi i j}{2^n} i} |212\rangle + \frac{1}{16}e^{42.33 \cdot \frac{2 \pi i j}{2^n} i} |213\rangle + \frac{1}{16}e^{43.33 \cdot \frac{2 \pi i j}{2^n} i} |214\rangle + \frac{1}{16}e^{44.33 \cdot \frac{2 \pi i j}{2^n} i} |215\rangle + \frac{1}{16}e^{45.33 \cdot \frac{2 \pi i j}{2^n} i} |216\rangle + \frac{1}{16}e^{46.33 \cdot \frac{2 \pi i j}{2^n} i} |217\rangle + \frac{1}{16}e^{47.33 \cdot \frac{2 \pi i j}{2^n} i} |218\rangle + \frac{1}{16}e^{48.33 \cdot \frac{2 \pi i j}{2^n} i} |219\rangle + \frac{1}{16}e^{49.33 \cdot \frac{2 \pi i j}{2^n} i} |220\rangle + \frac{1}{16}e^{50.33 \cdot \frac{2 \pi i j}{2^n} i} |221\rangle + \frac{1}{16}e^{51.33 \cdot \frac{2 \pi i j}{2^n} i} |222\rangle + \frac{1}{16}e^{52.33 \cdot \frac{2 \pi i j}{2^n} i} |223\rangle + \frac{1}{16}e^{53.33 \cdot \frac{2 \pi i j}{2^n} i} |224\rangle + \frac{1}{16}e^{54.33 \cdot \frac{2 \pi i j}{2^n} i} |225\rangle + \frac{1}{16}e^{55.33 \cdot \frac{2 \pi i j}{2^n} i} |226\rangle + \frac{1}{16}e^{56.33 \cdot \frac{2 \pi i j}{2^n} i} |227\rangle + \frac{1}{16}e^{57.33 \cdot \frac{2 \pi i j}{2^n} i} |228\rangle + \frac{1}{16}e^{58.33 \cdot \frac{2 \pi i j}{2^n} i} |229\rangle + \frac{1}{16}e^{59.33 \cdot \frac{2 \pi i j}{2^n} i} |230\rangle + \frac{1}{16}e^{60.33 \cdot \frac{2 \pi i j}{2^n} i} |231\rangle + \frac{1}{16}e^{61.33 \cdot \frac{2 \pi i j}{2^n} i} |232\rangle + \frac{1}{16}e^{62.33 \cdot \frac{2 \pi i j}{2^n} i} |233\rangle + \frac{1}{16}e^{63.33 \cdot \frac{2 \pi i j}{2^n} i} |234\rangle + \frac{1}{16}e^{64.33 \cdot \frac{2 \pi i j}{2^n} i} |235\rangle + \frac{1}{16}e^{65.33 \cdot \frac{2 \pi i j}{2^n} i} |236\rangle + \frac{1}{16}e^{66.33 \cdot \frac{2 \pi i j}{2^n} i} |237\rangle + \frac{1}{16}e^{67.33 \cdot \frac{2 \pi i j}{2^n} i} |238\rangle + \frac{1}{16}e^{68.33 \cdot \frac{2 \pi i j}{2^n} i} |239\rangle + \frac{1}{16}e^{69.33 \cdot \frac{2 \pi i j}{2^n} i} |240\rangle + \frac{1}{16}e^{70.33 \cdot \frac{2 \pi i j}{2^n} i} |241\rangle + \frac{1}{16}e^{71.33 \cdot \frac{2 \pi i j}{2^n} i} |242\rangle + \frac{1}{16}e^{72.33 \cdot \frac{2 \pi i j}{2^n} i} |243\rangle + \frac{1}{16}e^{73.33 \cdot \frac{2 \pi i j}{2^n} i} |244\rangle + \frac{1}{16}e^{74.33 \cdot \frac{2 \pi i j}{2^n} i} |245\rangle + \frac{1}{16}e^{75.33 \cdot \frac{2 \pi i j}{2^n} i} |246\rangle + \frac{1}{16}e^{76.33 \cdot \frac{2 \pi i j}{2^n} i} |247\rangle + \frac{1}{16}e^{77.33 \cdot \frac{2 \pi i j}{2^n} i} |248\rangle + \frac{1}{16}e^{78.33 \cdot \frac{2 \pi i j}{2^n} i} |249\rangle + \frac{1}{16}e^{79.33 \cdot \frac{2 \pi i j}{2^n} i} |250\rangle + \frac{1}{16}e^{80.33 \cdot \frac{2 \pi i j}{2^n} i} |251\rangle + \frac{1}{16}e^{81.33 \cdot \frac{2 \pi i j}{2^n} i} |252\rangle + \frac{1}{16}e^{82.33 \cdot \frac{2 \pi i j}{2^n} i} |253\rangle + \frac{1}{16}e^{83.33 \cdot \frac{2 \pi i j}{2^n} i} |254\rangle + \frac{1}{16}e^{84.33 \cdot \frac{2 \pi i j}{2^n} i} |255\rangle$

**解答**

この回路は{doc}`後の実習 <qft>`で登場する量子フーリエ変換と深く関係しますが、量子フーリエ変換では任意の状態$\ket{j}$を上の状態に変換するのに対し、この問題では$j$は外から与えられているパラメータです。そのため回路もだいぶ簡単になり、実は制御ゲートなしで実装できます。

問題4と同じように、この状態も個々の量子ビットにおける$\ket{0}$と$\ket{1}$の重ね合わせの積として表現できます。そのためには$k$の二進数表現$k = \sum_{m=0}^{n-1} 2^m k_m \, (k_m=0,1)$を用いて、$\ket{k}$の位相を

$$
\exp\left(2\pi i \frac{jk}{2^n}\right) = \exp \left(2\pi i \frac{j}{2^n} \sum_{m=0}^{n-1} 2^m k_m\right) = \prod_{m=0}^{n-1} \exp \left(2\pi i \frac{j}{2^{n-m}} k_m\right)
$$

と分解します。一方

$$
\ket{k} = \ket{k_{n-1}} \otimes \ket{k_{n-2}} \otimes \cdots \ket{k_0}
$$

なので、

$$
e^{2\pi i jk/2^n} \ket{k} = e^{2\pi i jk_{n-1}/2} \ket{k_{n-1}} \otimes \cdots \otimes e^{2\pi i jk_{1}/2^{n-1}} \ket{k_1} \otimes e^{2\pi i jk_{0}/2^n} \ket{k_{0}}.
$$

最後に、$k_m = 0$ならば$e^{2\pi i jk_{m}/2^n} = 1$であることから、

$$
\frac{1}{\sqrt{2^n}}\sum_{k=0}^{2^n-1} e^{2\pi i jk/2^n} \ket{k} = \frac{1}{\sqrt{2}} \left(\ket{0} + e^{2\pi i j/2} \ket{1}\right) \otimes \cdots \otimes \frac{1}{\sqrt{2}} \left(\ket{0} + e^{2\pi i j/2^n} \ket{1}\right)
$$

がわかります。

```{code-block} python
j = 3
for m in range(num_qubits):
    circuit.h(m)
    circuit.p(2. * np.pi * j / (2 ** (num_qubits - m)), m)
```

## [課題]ダブルスリット実験のシミュレーション